# **The seventh in-class-exercise (20 points in total, 10/21/2020)**

Question description: In the last in-class-exercise (exercise-06), you collected the titles of 100 articles about data science, natural language processing, and machine learning. The 100 article titles will be used as the text corpus of this exercise. Perform the following tasks:

## (1) (8 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
!pip install pyLDAvis

In [2]:
# Write your code here
import pandas as pd
import nltk
# NLTK Stop words
from nltk.corpus import stopwords
from textblob import Word
nltk.download('wordnet')
from nltk.stem import PorterStemmer
st = PorterStemmer()
# spacy for lemmatization
import spacy
import re
import numpy as np
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('/content/titles.csv')
df.head()

,Titles
0,Foundations of statistical natural language pr...
1,Natural language processing with Python: analy...
2,The Stanford CoreNLP natural language processi...
3,Handbook of natural language processing
4,Natural language processing (almost) from scratch


In [4]:
# Write your code here

#1 Noise removal
df['Titles'] = df['Titles'].replace('\n','',regex = True)
df['Titles'] = df['Titles'].replace('[^\w\s]','',regex = True)

#2 Remove numbers
df['Titles'] = df['Titles'].str.replace('\d+','')
df['Titles'] = df['Titles'].str.replace('\d+','')
#3 Remove StopWords
stop_Words = ["i", "me", "my", "myself", "we", "our", "ours", 
              "ourselves", "you", "your", "yours", "yourself",
              "yourselves", "he", "him", "his", "himself", "she", "her",
              "hers", "herself", "it", "its", "itself", "they", "them",
              "their", "theirs", "themselves", "what", "which", "who",
              "whom", "this", "that", "these", "those", "am", "is",
              "are", "was", "were", "be", "been", "being", "have",
              "has", "had", "having", "do", "does", "did", "doing",
              "a", "an", "the", "and", "but", "if", "or", "because",
              "as", "until", "while", "of", "at", "by", "for", "with",
              "about", "against", "between", "into", "through",
              "during", "before", "after", "above", "below", "to",
              "from", "up", "down", "in", "out", "on", "off", "over",
              "under", "again", "further", "then", "once", "here",
              "there", "when", "where", "why", "how", "all", "any",
              "both", "each", "few", "more", "most", "other", "some",
              "such", "no", "nor", "not", "only", "own", "same", "so",
              "than", "too", "very", "s", "t", "can", "will", "just",
              "don", "should", "now"]
# stop = stopwords.words('english')
df['Titles'] = df['Titles'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_Words))
#4 Lowercase all text
df['Titles'] = df['Titles'].str.lower()
#5 Stemming
df['Titles'] = df['Titles'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#6 Lemmatization
df['Titles']=df['Titles'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [5]:
df.head()

,Titles
0,foundat statist natur languag process
1,natur languag process python analyz text natur...
2,the stanford corenlp natur languag process too...
3,handbook natur languag process
4,natur languag process almost scratch


In [6]:
data = df.Titles.values.tolist()

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['foundat', 'statist', 'natur', 'languag', 'process']]


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=2) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=2)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example

print(trigram_mod[bigram_mod[data_words[1]]])

['natur_languag_process', 'python', 'analyz', 'text', 'natur_languag', 'toolkit']


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [9]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
  """https://spacy.io/api/annotation"""
  texts_out = []
  for sent in texts:
      doc = nlp(" ".join(sent)) 
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return texts_out

In [10]:
data_words_trigrams = make_trigrams(data_words)
data_words_bigrams = make_bigrams(data_words)

In [11]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])


In [12]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_trigrams = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_bigrams = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized_bigrams[:10])

[['process'], ['process', 'text', 'toolkit'], ['corenlp', 'process', 'toolkit'], ['process'], ['process', 'almost', 'scratch'], ['approach', 'process'], ['process'], ['read', 'process'], ['recent', 'trend', 'deep', 'learn', 'base', 'process'], ['process', 'deep', 'neural', 'network', 'multitask', 'learn']]


## (2) (8 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [13]:
# Write your code here



## (3) (4 points) Compare the results generated by the two topic modeling algorithms, which one is better? You should explain the reasons in details.

In [14]:
# Write your answer here (no code needed for this question)